white: //*[@id="radix-:r3:"]/div/div/div/div[3]/div/div/div[2]/div/div/ul/li[1]/button
red: //*[@id="radix-:r3:"]/div/div/div/div[3]/div/div/div[2]/div/div/ul/li[7]/button

In [ ]:
import os
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import ssl
import requests

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("window-size=1920x1080")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.implicitly_wait(3)
ssl._create_default_https_context = ssl._create_unverified_context
driver.get("https://www.musinsa.com/main/musinsa/recommend")

driver.find_element(By.XPATH, '/html/body/section/div[2]/button').click() # 팝업 창 닫기
driver.find_element(By.XPATH, '//*[@id="commonLayoutHeader"]/section/div[2]/div/button[1]').click() # 검색 창 들어가기
search_box = driver.find_element(By.XPATH, '//*[@id="commonSearchHome"]/main/header/div/input')  # 검색창의 XPATH를 사용
keyword = input("제품 검색어: ")
search_box.send_keys(keyword)  # 검색어 입력
driver.find_element(By.XPATH, '//*[@id="commonSearchHome"]/main/header/div/button[2]').click() # 돋보기 검색 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="commonLayoutContents"]/div/div[3]/div/div[1]/button[2]/span/span').click() # 컬러 창
driver.find_element(By.XPATH, '//*[@id="radix-:r3:"]/div/div/div/div[3]/div/div/div[2]/div/div/ul/li[7]/button').click() # 색상 선택
driver.find_element(By.XPATH, '//*[@id="radix-:r3:"]/div/div/div/div[5]/button').click() # 상품 보기

# 페이지 로딩 대기
time.sleep(5)  # 페이지가 완전히 로드될 때까지 기다림

# 스크롤 내려서 추가 이미지 로드
for _ in range(30):  # 스크롤 반복
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(3)

# 이미지 영역 선택
image_section = driver.find_element(By.XPATH, '//*[@id="commonLayoutContents"]/div/div[4]')

# 이미지들 선택
images = image_section.find_elements(By.XPATH, './/div/div/div/div/div/a/div/img')

# 이미지 URL 추출
img_urls = [img.get_attribute("data-original") or img.get_attribute("src") for img in images if img.get_attribute("data-original") or img.get_attribute("src")]

# 최대 50개만 선택
img_urls = img_urls[:50]

# 폴더 생성
if not os.path.exists("red_images"):
    os.makedirs("red_images")

# 헤더 추가 (요청 차단 방지)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# 이미지 저장
for i, img_url in enumerate(img_urls, start=1):
    response = requests.get(img_url, headers=headers)
    print(f"Request to {img_url} returned status code: {response.status_code}")
    if response.status_code == 200:
        with open(f"red_images/red_{i}.jpg", "wb") as file:
            file.write(response.content)
        print(f"Saved: red_{i}.jpg")
    else:
        print(f"Failed to download image {i}")

# 브라우저 종료
driver.quit()

print("이미지 저장 완료!")